<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/Cubic_Crystal_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook runs the jupyter-notebook given here on Colab.

In [1]:
!pip install m3gnet alignn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.1/285.1 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 45.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.8/973.8 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.0/265.0 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [7]:
import traceback
import warnings

import numpy as np
import pandas as pd
from pymatgen.core import Composition, Lattice, Structure
#from pymatgen.ext.matproj import MPRester
from pymatgen.ext.matproj import _MPResterLegacy as MPRester
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from tqdm import tqdm

from m3gnet.models import Relaxer

for category in (UserWarning, DeprecationWarning):
    warnings.filterwarnings("ignore", category=category, module="tensorflow")
    warnings.filterwarnings("ignore", category=category, module="pymatgen")

In [75]:
data = pd.read_html("http://en.wikipedia.org/wiki/Lattice_constant")[0]

In [78]:
data

,Material,Lattice constant (Å),Crystal structure,Ref.
0,C (diamond),3.567,Diamond (FCC),[7]
1,C (graphite),a = 2.461 c = 6.708,Hexagonal,NaN
2,Si,5.431020511,Diamond (FCC),[8][9]
3,Ge,5.658,Diamond (FCC),[8]
4,AlAs,5.6605,Zinc blende (FCC),[8]
...,...,...,...,...
86,CaVO3,3.767,Cubic perovskite,[15]
87,BaMnO3,a = 5.673 c = 4.71,Hexagonal,[15]
88,CaMnO3,a = 5.27 b = 5.275 c = 7.464,Orthorhombic perovskite,[15]
89,SrRuO3,a = 5.53 b = 5.57 c = 7.85,Orthorhombic perovskite,[15]


In [79]:

data = data[
    ~data["Crystal structure"].isin(
        ["Hexagonal", "Wurtzite", "Wurtzite (HCP)", "Orthorhombic", "Tetragonal perovskite", "Orthorhombic perovskite"]
    )
]
data.rename(columns={"Lattice constant (Å)": "a (Å)"}, inplace=True)

data["a (Å)"] = data["a (Å)"].map(float)


additional_fcc = """10 Ne 4.43 54 Xe 6.20
13 Al 4.05 58 Ce 5.16
18 Ar 5.26 70 Yb 5.49
20 Ca 5.58 77 Ir 3.84
28 Ni 3.52 78 Pt 3.92
29 Cu 3.61 79 Au 4.08
36 Kr 5.72 82 Pb 4.95
38 Sr 6.08 47 Ag 4.09
45 Rh 3.80 89 Ac 5.31
46 Pd 3.89 90 Th 5.08"""

additional_bcc = """3 Li 3.49 42 Mo 3.15
11 Na 4.23 55 Cs 6.05
19 K 5.23 56 Ba 5.02
23 V 3.02 63 Eu 4.61
24 Cr 2.88 73 Ta 3.31
26 Fe 2.87 74 W 3.16
37 Rb 5.59 41 Nb 3.30"""
def add_new(str_, structure_type, df):
    tokens = str_.split()
    new_crystals = []
    for i in range(int(len(tokens) / 3)):
        el = tokens[3 * i + 1].strip()
        if el not in df["Material"].values:
            new_crystals.append([tokens[3 * i + 1], structure_type, float(tokens[3 * i + 2])])
    df2 = pd.DataFrame(new_crystals, columns=data.columns)
    return pd.concat([df, df2])


# data = add_new(additional_fcc, "FCC", data)
# data = add_new(additional_bcc, "BCC", data)
# data.set_index("Material", inplace=True)
print(data)

       Material     a (Å)  Crystal structure    Ref.
0   C (diamond)  3.567000      Diamond (FCC)     [7]
2            Si  5.431021      Diamond (FCC)  [8][9]
3            Ge  5.658000      Diamond (FCC)     [8]
4          AlAs  5.660500  Zinc blende (FCC)     [8]
5           AlP  5.451000  Zinc blende (FCC)     [8]
..          ...       ...                ...     ...
79        KTaO3  3.988500   Cubic perovskite    [15]
81       SrTiO3  3.988050   Cubic perovskite    [15]
84       EuTiO3  7.810000   Cubic perovskite    [15]
85        SrVO3  3.838000   Cubic perovskite    [15]
86        CaVO3  3.767000   Cubic perovskite    [15]

[73 rows x 4 columns]


<ipython-input-79-08c8062c8bd3>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns={"Lattice constant (Å)": "a (Å)"}, inplace=True)
<ipython-input-79-08c8062c8bd3>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["a (Å)"] = data["a (Å)"].map(float)


Setting 5 Angstrom lattice constant to all crystals

In [97]:
MAPI_KEY_LEGACY = '' #Optional, keep your MAPI_LEGACY_KEY here
predicted = []
mp = []
mpr = MPRester(MAPI_KEY_LEGACY)
relaxer = Relaxer()  # This loads the default pre-trained model

# warnings.filterwarnings(action="ignore", category=UserWarning, module="tensorflow")
currently_m3gnet_failing = ['CsCl','CsI','Fe','Mo','W','V','Nb','Ta','Ne','CaVO3','Xe','Ar','Yb','Kr','Sr','Ac','Li','Na','Cs','K','Ba','Eu','Cr','Rb',"NC0.99"]

#for formula, v in tqdm(data.iterrows(), total=len(data)):
for ii,i in tqdm(data.iterrows(), total=len(data)):
  #M3gnet failing so skipping these
  formula=i['Material']
  if formula not in currently_m3gnet_failing: 
    formula = formula.split()[0]
    c = Composition(formula)
    els = sorted(c.elements)
    #cs = v["Crystal structure"]
    cs = i["Crystal structure"]
    # We initialize all the crystals with an arbitrary lattice constant of 5 angstroms.
    if "Zinc blende" in cs:
        s = Structure.from_spacegroup("F-43m", Lattice.cubic(5), [els[0], els[1]], [[0, 0, 0], [0.25, 0.25, 0.75]])
    elif "Halite" in cs:
        s = Structure.from_spacegroup("Fm-3m", Lattice.cubic(5), [els[0], els[1]], [[0, 0, 0], [0.5, 0, 0]])
    elif "Caesium chloride" in cs:
        s = Structure.from_spacegroup("Pm-3m", Lattice.cubic(5), [els[0], els[1]], [[0, 0, 0], [0.5, 0.5, 0.5]])
    elif "Cubic perovskite" in cs:
        s = Structure(
            Lattice.cubic(5),
            [els[0], els[1], els[2], els[2], els[2]],
            [[0.0, 0.0, 0.0], [0.5, 0.5, 0.5], [0.5, 0.5, 0], [0.0, 0.5, 0.5], [0.5, 0, 0.5]],
        )
    elif "Diamond" in cs:
        s = Structure.from_spacegroup("Fd-3m", Lattice.cubic(5), [els[0]], [[0.25, 0.75, 0.25]])
    elif "BCC" in cs:
        s = Structure(Lattice.cubic(5), [els[0]] * 2, [[0.0, 0.0, 0.0], [0.5, 0.5, 0.5]])
    elif "FCC" in cs:
        s = Structure(Lattice.cubic(5), [els[0]] * 4, [[0.0, 0.0, 0.0], [0.5, 0.5, 0], [0.0, 0.5, 0.5], [0.5, 0, 0.5]])
    else:
        predicted.append(0)
        mp.append(0)
        continue
    print(s)
    relax_results = relaxer.relax(s)

    final_structure = relax_results["final_structure"]

    predicted.append(final_structure.lattice.a)
    print('\nInitial,final,formula',5,final_structure.lattice.a,formula)
    #break
    try:
        mids = mpr.get_entries(s.composition.reduced_formula)
        for i in mids:
            try:
                structure = mpr.get_structure_by_material_id(i.entry_id)
                sga = SpacegroupAnalyzer(structure)
                sga2 = SpacegroupAnalyzer(final_structure)
                if sga.get_space_group_number() == sga2.get_space_group_number():
                    conv = sga.get_conventional_standard_structure()
                    mp.append(conv.lattice.a)
                    break
            except Exception:
                pass
        else:
            raise RuntimeError
    except Exception:
        mp.append(0)
        traceback.print_exc()


  0%|          | 0/73 [00:00<?, ?it/s]

Full Formula (C8)
Reduced Formula: C
abc   :   5.000000   5.000000   5.000000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (8)
  #  SP       a     b     c
---  ----  ----  ----  ----
  0  C     0.25  0.25  0.75
  1  C     0     0.5   0
  2  C     0.75  0.75  0.75
  3  C     0.5   0.5   0.5
  4  C     0.75  0.25  0.25
  5  C     0.5   0     0
  6  C     0     0     0.5
  7  C     0.25  0.75  0.25



Initial,final,formula 5 3.5718039756536863 C


  1%|▏         | 1/73 [00:16<19:25, 16.18s/it]

Full Formula (Si8)
Reduced Formula: Si
abc   :   5.000000   5.000000   5.000000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (8)
  #  SP       a     b     c
---  ----  ----  ----  ----
  0  Si    0.25  0.25  0.75
  1  Si    0     0.5   0
  2  Si    0.75  0.75  0.75
  3  Si    0.5   0.5   0.5
  4  Si    0.75  0.25  0.25
  5  Si    0.5   0     0
  6  Si    0     0     0.5
  7  Si    0.25  0.75  0.25

Initial,final,formula 5 5.450023339662936 Si


...

 96%|█████████▌| 70/73 [01:58<00:03,  1.12s/it]

Full Formula (Eu1 Ti1 O3)
Reduced Formula: EuTiO3
abc   :   5.000000   5.000000   5.000000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (5)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Eu    0    0    0
  1  Ti    0.5  0.5  0.5
  2  O     0.5  0.5  0
  3  O     0    0.5  0.5
  4  O     0.5  0    0.5

Initial,final,formula 5 3.9294272293452503 EuTiO3


 97%|█████████▋| 71/73 [02:00<00:02,  1.26s/it]

Full Formula (Sr1 V1 O3)
Reduced Formula: SrVO3
abc   :   5.000000   5.000000   5.000000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (5)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Sr    0    0    0
  1  V     0.5  0.5  0.5
  2  O     0.5  0.5  0
  3  O     0    0.5  0.5
  4  O     0.5  0    0.5

Initial,final,formula 5 3.906043550979565 SrVO3


100%|██████████| 73/73 [02:01<00:00,  1.67s/it]


In [98]:
data_1 = data[~data["Material"].isin(currently_m3gnet_failing)]

In [99]:
data_1

,Material,a (Å),Crystal structure,Ref.
0,C (diamond),3.567000,Diamond (FCC),[7]
2,Si,5.431021,Diamond (FCC),[8][9]
3,Ge,5.658000,Diamond (FCC),[8]
4,AlAs,5.660500,Zinc blende (FCC),[8]
5,AlP,5.451000,Zinc blende (FCC),[8]
...,...,...,...,...
77,ScN,4.520000,Halite,[14]
79,KTaO3,3.988500,Cubic perovskite,[15]
81,SrTiO3,3.988050,Cubic perovskite,[15]
84,EuTiO3,7.810000,Cubic perovskite,[15]


In [100]:
data_1["MP a (Å)"] = mp
data_1["Predicted a (Å)"] = predicted

<ipython-input-100-5bf6779e9a9e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1["MP a (Å)"] = mp
<ipython-input-100-5bf6779e9a9e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1["Predicted a (Å)"] = predicted


In [101]:
data_1


,Material,a (Å),Crystal structure,Ref.,MP a (Å),Predicted a (Å)
0,C (diamond),3.567000,Diamond (FCC),[7],3.573710,3.571804
2,Si,5.431021,Diamond (FCC),[8][9],5.468728,5.450023
3,Ge,5.658000,Diamond (FCC),[8],5.762862,5.769803
4,AlAs,5.660500,Zinc blende (FCC),[8],5.733756,5.730271
5,AlP,5.451000,Zinc blende (FCC),[8],5.507112,5.503462
...,...,...,...,...,...,...
77,ScN,4.520000,Halite,[14],4.518310,4.517974
79,KTaO3,3.988500,Cubic perovskite,[15],4.030839,4.032654
81,SrTiO3,3.988050,Cubic perovskite,[15],3.945130,3.944807
84,EuTiO3,7.810000,Cubic perovskite,[15],3.961195,3.929427


In [104]:
from sklearn.metrics import mean_absolute_error
print('MAE wrt Exp/Wiki data',mean_absolute_error(data_1['a (Å)'],data_1['Predicted a (Å)']))
print('MAE wrt MP data',mean_absolute_error(data_1['MP a (Å)'],data_1['Predicted a (Å)']))

MAE wrt Exp/Wiki data 0.13163907193121901
MAE wrt MP data 0.0747479481970251
